# Natural Algorithms Genetic algorithm


In [1]:
import pandas as pd
import numpy as np
import random
import csv

# 1 Fitness function

In [55]:
def getFitness(population):
    fitnessofpopulation = []
    runtimeofeverymachineinachromosome = []
    for machine in range(machinecount):
        runtimeofeverymachineinachromosome.append(0)
    for state in population:
        for machine in range(machinecount):
            runtimeofeverymachineinachromosome[machine] = 0
        for job in range(len(state)-1):
            runtimeofeverymachineinachromosome[state[job]] -= int(values[job][0])
        fitnessofpopulation.append(min(runtimeofeverymachineinachromosome))  
    correction = min(fitnessofpopulation)-1
    index = 0
    for fitness in fitnessofpopulation:
        fitnessofpopulation[index] = fitness - correction
        index+=1
    return fitnessofpopulation

In [61]:
def getBestRuntime(population):
    fitnessofpopulation = []
    runtimeofeverymachineinachromosome = []
    for machine in range(machinecount):
        runtimeofeverymachineinachromosome.append(0)
    for state in population:
        for machine in range(machinecount):
            runtimeofeverymachineinachromosome[machine] = 0
        for job in range(len(state)-1):
            runtimeofeverymachineinachromosome[state[job]] -= int(values[job][0])
        fitnessofpopulation.append(min(runtimeofeverymachineinachromosome))
    return abs(min(fitnessofpopulation))

# 2 Initializer

In [3]:
# This method calls the random or heuristic initializer

def initialize(init_method):
    if init_method == "random":
        pop = initialize_random()
    elif init_method == "heuristic":
        pop = initialize_heuristic()

    return pop

In [4]:
# This method initializes a population with random chromosomes

def initialize_random():
    pop = []
    for chromosome in range(popsize):
        # initialize empty list which represents the chromosome
        jobs = []
        # assign a random machine to each job    
        for job in range(jobcount):
            jobs.append(random.randint(0,machinecount-1))
    
        pop.append(jobs)
    return pop

## 2.1 Intialize heuristic

In [5]:
# This method initializes a population with heuristic chromosomes
# For every chromosome: We use the ratio between jobs/machines to assign every machine to a nearly identical number of jobs.
# If we assume that all jobs need the same processing time, this would already yield a nearly perfect solution.

def initialize_heuristic():
    pop = []

    for chromosome in range(popsize):
        jobs = []
        machines = []
        not_heuristic = True

        #create list that serves to count how many jobs a machine is assigned to
        for machine in range(machinecount):
            machines.append(0)
    
        ratio = int(jobcount/machinecount)

        #assign a random machine to each job and count how often each machine is assigned
        for job in range(jobcount):
            machine = random.randint(0,machinecount-1)
            jobs.append(machine)
            machines[machine] += 1

        #change the machine assignment such that every machine is assigned a nearly identical number of jobs
        while not_heuristic:
            #i is used to count machines that dont have enough jobs assigned to them, given by the ratio
            i = 0
            #used as machine index
            machine = 0
    
            #check for every machine
            for counter in machines:
                #if it has enough jobs assigned, given by the ratio
                if counter < ratio:
                    #if it doesnt, we assign it to one more random job 
                    jobs[random.randint(0,jobcount-1)] = machine
                    i += 1     
                machine += 1
        
            #empty the counter list    
            machines.clear()
            for machine in range(machinecount):
                machines.append(0)
    
            #count again how often each machine is assigned 
            for job in jobs:
                machines[job] += 1
    
            #stop if there are no more machines that dont have enough jobs
            if i == 0:    
                not_heuristic = False 
    
        pop.append(jobs)

    return pop     

# 3 Selection

In [6]:
def select(sel_method, population):
    matingPool = []
    if(sel_method == "fitness_proportionate"):
        #print("Hello")
        run = 0
        while run in range(matingsize):
            selected = fitness_proportionate(population)
            if selected not in matingPool:
                matingPool.append(selected)
                run+=1
    elif(sel_method == "truncated"):
        #print("Hello1")
        matingPool = truncated(population, int(popsize*0.25))
    elif(sel_method == "tournament"):
        #print("Hello2")
        run = 0
        while run in range(matingsize):
            selected = tournament(population, int(popsize*0.25))
            if selected not in matingPool:
                matingPool.append(selected)
                run+=1
    return matingPool

In [7]:
def fitness_proportionate(population):
    probabilityList = []
    fitnessList = getFitness(population)
    comulatedFit = sum(fitnessList)
    for fitness in fitnessList:
        probabilityList.append((fitness/comulatedFit))
    comulatedProb = sum(probabilityList)
    
    return population[int(np.random.choice(popsize, 1, probabilityList))]

In [8]:
def truncated(population, s):
    fitnessList = getFitness(population)
    matingPool = []
    for run in range(int(popsize/s)):
        fittest = fitnessList.index(max(fitnessList))
        for i in range(s):
            matingPool.append(population[fittest])
        fitnessList[fittest] = 0
        
    return matingPool
        
        

In [9]:
def tournament(population, s):
    chosenStates = []
    chosenNum = np.random.choice(popsize, s)
    for num in chosenNum:
        chosenStates.append(population[num])
    fittest = getFitness(chosenStates).index(max(getFitness(chosenStates)))

    return chosenStates[fittest]

# 4 Recombination Algorithms

In [10]:
def recombine(rec_method, mating_pool):
    """Calls wanted recombine method"""
    if rec_method == 'k-point_crossover':
        offspring = kpoint(mating_pool)
        
    if rec_method == 'uniform_crossover':
        offspring = uniformorderbased(mating_pool)
        
    return offspring

## 4.1 k-point crossover

In [11]:
# k-point crossover

def kpoint(population):
    
    # generating k
    k = random.randint(1, len(population[0])-1)
    
    # generating crossover
    crossover = []
    for i in range(k):
        crossover.append(random.randint(1, len(population[0])-1))

    # Generating childs aka new population
    result = []
    switch = True
    for parent in range(1, len(population), 2):
        counter = 0
        child1 = []
        child2 = []
        for elementp1, elementp2 in zip(population[parent - 1], population[parent]):
            if switch:
                child1.append(elementp1)
                child2.append(elementp2)
            else:
                child1.append(elementp2)
                child2.append(elementp1)
            counter += 1
            if counter in crossover:
                switch^=True
        result.append(child1)
        result.append(child2)
    
    return result

## 4.2 Uniform order based crossover

In [12]:
def uniformorderbased(population):
    
    # Generating Template
    template = []
    for i in range(len(population[0])):
        template.append(random.randint(0, 1))

    # Generating children aka new population
    result = []
    for parent in range(1, len(population), 2):
        counter = 0
        child1 = []
        child2 = []
        for elementp1, elementp2 in zip(population[parent - 1], population[parent]):
            if template[counter] == 1:
                child1.append(elementp1)
                child2.append(elementp2)
            else:
                child1.append(elementp2)
                child2.append(elementp1)
            counter += 1

        result.append(child1)
        result.append(child2)
    return result

# 5 Mutation

In [13]:
def mutate(mut_method, offspring):
    if mut_method == "bitflip":
        offspring = bitflip(offspring)
    if mut_method == "swap":
        offspring = swap(offspring)
        
    return offspring

In [14]:
def bitflip(offspring):
    state = offspring[random.randint(0, len(offspring)-1)]
    index = offspring.index(state)

    i = 0
    while i < len(state):
        state[i] = random.randint(0, machinecount - 1)
        i += 1
        
    offspring[index] = state    
    
    return offspring

In [15]:
def swap(offspring):
    state = offspring[random.randint(0, len(offspring)-1)]
    index = offspring.index(state)

    i = 0
    m = random.randInt(1, len(state)-1)
    while i < m:
        index1 = random.randint(0, len(state)-1)
        value1 = state[index1]
        index2 = random.randint(0, len(state)-1)
        value2 = state[index2]
        state[index1] = value2
        state[index2] = value1
        i += 1
        
    offspring[index] = state
    
    return offspring

# 6 Replacement

In [16]:
def replace(rep_method, population, offspring):
    if rep_method == "steadyState":
        pop = steadyState(population, offspring)
    elif rep_method == "dellAll":
        pop = dellAll(population, offspring)
    elif rep_method == "elitist":
        pop = elitist(population, offspring)
        
    return pop    

In [17]:
def runtimecounter(dataList,machineCount,population):
    subCount = 0
    for i in range(0,len(population)):
        # range(0,machinecount+1)
        for j in range(1,10):
            count = 0
            for x in range(0,len(dataList)):
                if j == population[i][x]:
                    count = count + dataList[x]
            if count > subCount:
                subCount = count
                identification = i
                
    return identification

In [18]:
# delete the worst chromosome
def steadyState(population, offspring):
    population.extend(offspring)
    return population.remove(runtimecounter(dataList,machineCount,population))

In [19]:
def dellAll(population, offspring):
    return offspring

In [41]:
def elitist(population, offspring):
    newPopulation = []
    population.extend(offspring)
    fitnessList = getFitness(population)
    for i in range(popsize):
        fittest = fitnessList.index(max(fitnessList))
        newPopulation.append(population[fittest])
        fitnessList[fittest] = 0
    return newPopulation

# 7 Main

## 7.1 Eingabe

In [26]:
eingabe = input("Welche benchmark soll laufen? ")
popsize = 20
matingsize = 10 
mutationprob = 0.1

with open(eingabe + ".csv") as f:
    reader = csv.reader(f)
    next(reader) # skip header
    values = []
    for row in reader:
        values.append(row)

eingabe = eingabe.split("_")

machinecount = int(eingabe[1])
jobcount = int(eingabe[2])

Welche benchmark soll laufen? benchmark1_20_300


## 7.2 Pseudocode

In [68]:
def main():
    population = initialize("random")
    termination = False
    i = 0
    while i != 1:
        mating_pool = select("fitness_proportionate", population)
        offspring = recombine("uniform_crossover", mating_pool)
        offspring = mutate("bitflip", offspring)
        population = replace("elitist", population, offspring)
        i = max(getFitness(population))     
        
    
    return getBestRuntime(population)

In [74]:
proc_times = []
for a in range(10):
    proc_times.append(main())

print(proc_times)

[6527, 6636, 6541, 6827, 6534, 6616, 6543, 6611, 6629, 6848]
